In [2]:
#%pip install torch --index-url https://download.pytorch.org/whl/cpu
%pip install pandas

/home/lanv/recipencoder/autoencoder/.venv/bin/python3.13: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# %pip install ollama

In [4]:
# %curl -L -o data/archive.zip https://www.kaggle.com/api/v1/datasets/download/saldenisov/recipenlg

In [5]:
import pandas as pd
import json

In [ ]:
df = pd.read_csv("full_dataset.csv") # read in recipesdata/dataset/full_dataset.csv

In [7]:
dfsm = df[:1000]

In [8]:
dfsm

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."
...,...,...,...,...,...,...,...
995,995,Heath Bar Pie,"[""3 Heath bars, chopped fine"", ""1 medium conta...","[""Mix chopped Heath bars with whipped topping ...",www.cookbooks.com/Recipe-Details.aspx?id=976718,Gathered,"[""graham cracker pie crust"", ""chocolate curls""]"
996,996,Victorian Baked French Toast,"[""1 c. brown sugar"", ""1/3 c. butter"", ""2 Tbsp....","[""Cook brown sugar, butter and corn syrup in s...",www.cookbooks.com/Recipe-Details.aspx?id=908190,Gathered,"[""brown sugar"", ""butter"", ""light corn syrup"", ..."
997,997,Quick Swedish Meatballs,"[""1 lb. ground beef"", ""1 c. soft bread crumbs""...","[""Combine meat, bread crumbs, cheese, soup mix...",www.cookbooks.com/Recipe-Details.aspx?id=850050,Gathered,"[""ground beef"", ""bread crumbs"", ""cream cheese""..."
998,998,Irish Stew(Microwave),"[""2 lb. lamb, cut in 1-inch cubes"", ""2 c. wate...","[""In 4-quart casserole, combine lamb, 1 1/4 cu...",www.cookbooks.com/Recipe-Details.aspx?id=1017368,Gathered,"[""lamb"", ""water"", ""onion soup"", ""bay leaf"", ""c..."


In [9]:
dfsm = dfsm["NER"].apply(lambda x: eval(x))

In [10]:
def clean(ing:str):
  out= ing.lower()
  out = out.strip()
  out = out.replace("-", " ")
  out = out.replace("_", " ")
  return out

In [11]:
ings = set()
for index, i in enumerate(dfsm):
  if(index % 10000 == 0):
    print(index)
  o = [clean(j) for j in i]
  ings = ings.union(set(o))


0


In [12]:
ings

{'brisket',
 'cold milk',
 'ham bouillon',
 'paprika',
 'cherries',
 'shredded monterey jack cheese',
 'breadcrumbs',
 'frozen mixed vegetables',
 'mixed nuts',
 'amount of mayonnaise',
 'coconut flavoring',
 'ground lean beef',
 "cap'n",
 'romano cheese',
 'cayenne red pepper',
 'orange flavor',
 'recipe basic sweet dough',
 'white minute rice',
 'corn flake crumbs',
 'fresh ground black pepper',
 'caramel sauce',
 'milk',
 'whipped topping',
 'bottles ginger ale',
 'red food coloring',
 'spanish rice',
 'northern beans',
 'cornflakes cereal',
 'brandy',
 'beef consomme',
 'ginger',
 'generous',
 'lemon supreme cake mix',
 'pasta',
 'corn meal',
 'instant tea',
 'unflavored gelatine',
 'dark molasses',
 'salad',
 'marshmallow cream',
 'green parsley',
 'ground round',
 'sesame seed',
 'green beans',
 'muffins',
 'light karo syrup',
 'cranberry juice cocktail',
 'nutmeg',
 'frozen artichoke',
 'vanilla ice cream',
 'coffee',
 'meaty soup bones',
 'rye bread',
 'olive oil',
 'squash',
 

In [13]:
print(len(ings))
reuse_fact = len(dfsm) / len(ings)
reuse_fact

1096


0.9124087591240876

In [14]:
from ollama import Client
client = Client(
  host='http://localhost:11435',
)

for model in client.list()["models"]:
  print(model["model"])

stable-code:3b
phi3:3.8b


In [15]:
out = client.embed(
  model='phi3:3.8b',
  input=list(ings)
)

In [16]:
display(out["embeddings"][0])
X = out["embeddings"]

[-0.011141346,
 0.045257628,
 0.02114121,
 0.047058504,
 0.018588912,
 0.025083756,
 -0.052866444,
 0.018442752,
 -0.00493127,
 0.036111355,
 -0.0111787515,
 0.04553375,
 -0.015155174,
 -0.014096309,
 0.037142493,
 -0.050672002,
 0.020808848,
 0.055116236,
 0.0025919583,
 0.037616234,
 -0.024082122,
 0.014485948,
 0.022280887,
 0.06825671,
 0.02529863,
 0.0108975135,
 -0.020290107,
 0.0056353225,
 -0.022508712,
 -0.019321807,
 0.012398659,
 -0.024065036,
 -0.016008403,
 0.016929442,
 0.032049257,
 -0.008641512,
 0.026047114,
 -0.05492865,
 0.009042946,
 -0.09032297,
 0.025448315,
 -0.09472241,
 -0.008185283,
 -0.010856392,
 -0.07423979,
 -0.006490769,
 0.024795258,
 -0.017440813,
 0.04624948,
 0.0628335,
 -0.0014524508,
 -0.017079331,
 0.013735871,
 -0.003917745,
 0.036098268,
 -0.0013044011,
 0.050454155,
 0.018343523,
 0.013032376,
 0.003425466,
 0.039751355,
 -0.009747373,
 -0.026281338,
 -0.10481878,
 0.024996785,
 -0.04853155,
 -0.007112187,
 0.009051859,
 -0.046242874,
 0.0104351

In [31]:
from sklearn.cluster import DBSCAN
db = DBSCAN(eps=0.05, min_samples=2).fit(X) # X is the embedding vector
labels = db.labels_


# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)


Estimated number of clusters: 2
Estimated number of noise points: 1082


In [32]:
from collections import defaultdict
names = list(ings)

# Assuming `labels` is the output from DBSCAN and `names` is the list of ingredient names
# Example:
# labels = db.labels_
# names = list(ings)

# Group ingredients by their cluster labels
clusters = defaultdict(list)
for idx, label in enumerate(labels):
    clusters[label].append(names[idx])

# Print members of each cluster
for cluster_id, members in clusters.items():
    if cluster_id == -1:
        print(f"Noise Points (Unclustered): {len(members)} items")
    else:
        print(f"Cluster {cluster_id}: {len(members)} items")
    print(members)
    print("-" * 40)

# Print summary
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print(f"Estimated number of clusters: {n_clusters_}")
print(f"Estimated number of noise points: {n_noise_}")

Noise Points (Unclustered): 1082 items
['brisket', 'cold milk', 'ham bouillon', 'paprika', 'cherries', 'shredded monterey jack cheese', 'breadcrumbs', 'frozen mixed vegetables', 'mixed nuts', 'amount of mayonnaise', 'coconut flavoring', 'ground lean beef', "cap'n", 'romano cheese', 'cayenne red pepper', 'orange flavor', 'recipe basic sweet dough', 'white minute rice', 'corn flake crumbs', 'fresh ground black pepper', 'caramel sauce', 'milk', 'whipped topping', 'bottles ginger ale', 'red food coloring', 'spanish rice', 'northern beans', 'cornflakes cereal', 'brandy', 'beef consomme', 'ginger', 'generous', 'lemon supreme cake mix', 'pasta', 'corn meal', 'instant tea', 'unflavored gelatine', 'dark molasses', 'salad', 'marshmallow cream', 'green parsley', 'ground round', 'sesame seed', 'green beans', 'muffins', 'light karo syrup', 'cranberry juice cocktail', 'nutmeg', 'frozen artichoke', 'vanilla ice cream', 'coffee', 'meaty soup bones', 'rye bread', 'olive oil', 'yellow food coloring', 'p

In [23]:
import numpy as np
import matplotlib.pyplot as plt
unique_labels = set(labels)
core_samples_mask = np.zeros_like(labels, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True

colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = labels == k

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(
        xy[:, 0],
        xy[:, 1],
        "o",
        markerfacecolor=tuple(col),
        markeredgecolor="k",
        markersize=14,
    )

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(
        xy[:, 0],
        xy[:, 1],
        "o",
        markerfacecolor=tuple(col),
        markeredgecolor="k",
        markersize=6,
    )

plt.title(f"Estimated number of clusters: {n_clusters_}")
plt.show()

TypeError: only integer scalar arrays can be converted to a scalar index

In [18]:
response = client.generate(
  # model='smollm2:1.7b',
  model='phi3:3.8b',
  options={"temperature":0.7},
  format="json",
  system="You are an efficient assistant. Respond in JSON format",
  # prompt="Given an ingredient, simplify it to the core ingredient. Ignore size or quality descriptions. Use one or two words: Item: {}.".format('big egg'))
  prompt="Describe the ingredient with base description of what this is and optional size and preparation descriptions. Raw ingredient name: {}.".format('bite size shredded rice biscuits'))

json.loads(response["response"])


{'ingredient': {'name': 'bite-size shredded coconut',
  'base_description': 'Shredded fresh or dried coconut, commonly used in baking and dessert recipes.',
  'optional_sizes': [{'small_batch': 'A small package of 1 cup (250g) shredded coconut.'},
   {'jumbo_size': 'A larger container holding up to 3 cups (710g) for special occasions or recipes requiring more volume.'}],
  'optional_preparation': [{'description': "The shredded coconut can be used as-is in baking, cooking, and desserts. If it's not pre-shredded, the user may need to grate or purchase ready-shredded."}]}}

In [19]:


json.loads(response["response"])


{'ingredient': {'name': 'bite-size shredded coconut',
  'base_description': 'Shredded fresh or dried coconut, commonly used in baking and dessert recipes.',
  'optional_sizes': [{'small_batch': 'A small package of 1 cup (250g) shredded coconut.'},
   {'jumbo_size': 'A larger container holding up to 3 cups (710g) for special occasions or recipes requiring more volume.'}],
  'optional_preparation': [{'description': "The shredded coconut can be used as-is in baking, cooking, and desserts. If it's not pre-shredded, the user may need to grate or purchase ready-shredded."}]}}